In [ ]:
https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/mcp/agent_as_mcp_server.py

In [ ]:
from dotenv import load_dotenv

import os

load_dotenv(override=True)

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
WEATHER_API_URL = os.getenv("WEATHER_API_URL")

In [4]:
import asyncio
from typing import Any

from agent_framework import AgentProtocol, AgentThread, HostedMCPTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

"""
Azure AI Agent with Hosted MCP Example

This sample demonstrates integration of Azure AI Agents with hosted Model Context Protocol (MCP)
servers, including user approval workflows for function call security.
"""


async def handle_approvals_with_thread(query: str, agent: "AgentProtocol", thread: "AgentThread"):
    """Here we let the thread deal with the previous responses, and we just rerun with the approval."""
    from agent_framework import ChatMessage

    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        for user_input_needed in result.user_input_requests:
            print(
                f"User Input Request for function from {agent.name}: {user_input_needed.function_call.name}"
                f" with arguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


async def main() -> None:
    """Example showing Hosted MCP tools for a Azure AI Agent."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # enable azure-ai observability
        await chat_client.setup_azure_ai_observability()
        agent = chat_client.create_agent(
            name="WeatherAgent",
            instructions="You are a helpful assistant that can get weather information.",
            tools=HostedMCPTool(
                name="get_forecast",
                url=WEATHER_API_URL,
                headers={"x-api-key": WEATHER_API_KEY},
            ),
        )
        thread = agent.get_new_thread()
        # First query
        query1 = "Whats the weather in Seattle?"
        print(f"User: {query1}")
        result1 = await handle_approvals_with_thread(query1, agent, thread)
        print(f"{agent.name}: {result1}\n")
        print("\n=======================================\n")
        # Second query
        query2 = "What is Microsoft Agent Framework?"
        print(f"User: {query2}")
        result2 = await handle_approvals_with_thread(query2, agent, thread)
        print(f"{agent.name}: {result2}\n")

await main()


[2025-11-18 08:52:23 - c:\Users\memasanz\repos\agent-framework-exploration\.venv\Lib\site-packages\azure\monitor\opentelemetry\exporter\statsbeat\_manager.py:189 - WARNING] Statsbeat re-initialized with the same configuration.
[2025-11-18 08:52:23 - c:\Users\memasanz\repos\agent-framework-exploration\.venv\Lib\site-packages\azure\monitor\opentelemetry\exporter\statsbeat\_manager.py:189 - WARNING] Statsbeat re-initialized with the same configuration.


User: Whats the weather in Seattle?
User Input Request for function from WeatherAgent: get_forecast with arguments: {"latitude":47.6062,"longitude":-122.3321}
WeatherAgent: Here’s the current weather forecast for Seattle:

- **Overnight:** Mostly cloudy, low around 43°F. Light south wind. Slight chance of rain (less than a tenth of an inch).
- **Tuesday:** Cloudy with a 30% chance of rain before 2pm. High near 49°F, falling to 47°F in the afternoon. Light east-southeast wind.
- **Tuesday Night:** Partly cloudy, low around 38°F, warming to 40°F overnight. Light north-northeast wind.
- **Wednesday:** Mostly sunny, high near 52°F. Light northeast wind.
- **Wednesday Night:** Mostly cloudy, low around 43°F. 30% chance of rain after 10pm. Moderate east-southeast wind.

Let me know if you want a more detailed or extended forecast!



User: What is Microsoft Agent Framework?
WeatherAgent: The **Microsoft Agent Framework** is a set of tools, libraries, and APIs developed by Microsoft to enable